In [0]:
%run "./Configuration"

In [0]:
# Define the metadata widget:
dbutils.widgets.text("entityName", "")
dbutils.widgets.text("entitySourceQuery", "")
dbutils.widgets.text("entityColumns", "")
dbutils.widgets.text("sourceSystem", "")

# Save the parameter to a variable:
entityName = dbutils.widgets.get('entityName')
connection = json.loads(dbutils.widgets.get('entitySourceQuery'))
sourceSystemName = dbutils.widgets.get('sourceSystem')

# Access query from connection:
query = connection['sourceQuery']

# Access url from connection:
url = connection['connectionString']

In [0]:
# Check source system:
if sourceSystemName == 'TotesysDB':
  
  # If entity is date, do not ingest:
  if entityName == 'date':
    dbutils.notebook.exit('None')
  
  else:
    # Query the totesys database to ingest data:
    df = query_totesys(query)

In [0]:
# Check source system:
if sourceSystemName == 'AdventureWorks':
    # URL comes from sourceQuery:
    url = connection['connectionString']

    # Get the CSV file content as a string:
    response = requests.get(url)
    
    # Define temp path in DBFS:
    temp_path = f'/dbfs/tmp/{entityName}.csv'
    
    # Write to DBFS:
    dbutils.fs.put(temp_path, response.content.decode('utf-8'), overwrite=True)
    
    # Create a DataFrame from the CSV file:
    df = spark.read.csv(temp_path, header=True, inferSchema=True)

In [0]:
# Check data exists:
if df.count() > 0:
    # Define file_path to save data:
    file_path = f"/{sourceSystemName}/{entityName}/{date_path}/"
    
    # Set location for parquet:
    location = BRONZE + file_path
    
    # Write parquet to data lake:
    df.write.mode('overwrite').format('parquet').option('header', 'true').save(location)


In [0]:
# Output file path for entity:
dbutils.notebook.exit(file_path) 